In [18]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import xgboost as xgb
import seaborn as sns

In [39]:
train = pd.read_csv("./data/train.csv", encoding="utf-8")
test = pd.read_csv("./data/test.csv", encoding="utf-8")

le = LabelEncoder()
train['IsHoliday'] = le.fit_transform(train['IsHoliday'])
test['IsHoliday'] = le.fit_transform(test['IsHoliday'])

train[['day', 'month', 'year']] = train['Date'].str.split('/', expand=True).astype(int)
test[['day', 'month', 'year']] = test['Date'].str.split('/', expand=True).astype(int)

test

,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,day,month,year
0,1,1,05/10/2012,68.55,3.617,8077.89,NaN,18.22,3617.43,3626.14,6.573,0,5,10,2012
1,2,1,12/10/2012,62.99,3.601,2086.18,NaN,8.11,602.36,5926.45,6.573,0,12,10,2012
2,3,1,19/10/2012,67.97,3.594,950.33,NaN,4.93,80.25,2312.85,6.573,0,19,10,2012
3,4,1,26/10/2012,69.16,3.506,2585.85,31.75,6.00,1057.16,1305.01,6.573,0,26,10,2012
4,5,2,05/10/2012,70.27,3.617,6037.76,NaN,10.04,3027.37,3853.40,6.170,0,5,10,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,176,44,26/10/2012,46.97,3.755,38.65,2.61,0.98,NaN,457.74,5.217,0,26,10,2012
176,177,45,05/10/2012,64.89,3.985,5046.74,NaN,18.82,2253.43,2340.01,8.667,0,5,10,2012
177,178,45,12/10/2012,54.47,4.000,1956.28,NaN,7.89,599.32,3990.54,8.667,0,12,10,2012
178,179,45,19/10/2012,56.47,3.969,2004.02,NaN,3.18,437.73,1537.49,8.667,0,19,10,2012


In [6]:
def splitStore(train, test, i):
    train = train[train.Store == i]
    test = test[test.Store == i]
    
    train = train.reset_index()
    train1 = train.iloc[100:]
    vali = train.iloc[88:91]

    return train1, test, vali


def reArrange(train, test, vali):
    xtrain = train[['Temperature', 'Fuel_Price','Promotion3', 'month']]
    ytrain = train[["Weekly_Sales"]]
    xvali = vali[['Temperature', 'Fuel_Price','Promotion3', 'month']]
    yvali = vali[["Weekly_Sales"]]
    xtest = test[['Temperature', 'Fuel_Price', 'Promotion3', 'month']]
    
    return xtrain, ytrain, xtest, xvali, yvali

def scaling(xtrain, xtest, xvali):
    scaler = MinMaxScaler()
    xtrain = pd.DataFrame(scaler.fit_transform(xtrain))
    xtest = pd.DataFrame(scaler.fit_transform(xtest))
    xvali = pd.DataFrame(scaler.fit_transform(xvali))
    
    return xtrain, xtest, xvali

In [7]:
def xgbModel(xtrain, ytrain, xtest, xvali, yvali):
    eval_set = [(xvali, yvali)]
    xgb_model = xgb.XGBRegressor(n_estimators=500, learning_rate=0.05, subsample=0.7,
                           colsample_bytree=0.7, max_depth=5, silent = 1, nthread = 4, min_child_weight = 4)
    
    xgb_model.fit(xtrain,ytrain, eval_set=eval_set, eval_metric="rmse", early_stopping_rounds=20)
    pred = xgb_model.predict(xtest)
    
    return pred

In [8]:
def splitStore2(train, test, i):
    train = train[train.Store == i]
    test = test[test.Store == i]

    train = train.reset_index()
    train1 = train.iloc[100:]
    
    vali = train[train.month == 10]

    return train1, test, vali


def reArrange2(train, test, vali):
    xtrain = train[['Temperature', 'Fuel_Price', 'Unemployment', 'IsHoliday','month']]
    ytrain = train[["Weekly_Sales"]]
    xtest = test[['Temperature', 'Fuel_Price', 'Unemployment' ,'IsHoliday', 'month']]
    xvali = vali[['Temperature', 'Fuel_Price', 'Unemployment', 'IsHoliday','month']]
    yvali = vali[["Weekly_Sales"]]

    return xtrain, ytrain, xtest, xvali, yvali

def scaling2(xtrain, xtest, xvali):
    scaler = MinMaxScaler()
    xtrain = pd.DataFrame(scaler.fit_transform(xtrain))
    xtest = pd.DataFrame(scaler.fit_transform(xtest))
    xvali = pd.DataFrame(scaler.fit_transform(xvali))
    
    return xtrain, xtest, xvali

In [10]:
def randomForestModel(xtrain, ytrain, xtest):
    model = RandomForestRegressor()
    model.fit(xtrain,ytrain)
    pred = model.predict(xtest)
    
    return pred

In [11]:
result = []
exc = [36, 38 , 42, 43, 44] ##비 선형 지점
for i in range(1, 46):
    if i in exc:
        print(i)
        train1, test1, vali = splitStore2(train, test, i)
        xtrain, ytrain, xtest, xvali, yvali = reArrange2(train1, test1, vali)
        xtrain, xtest, xvali = scaling2(xtrain, xtest, xvali)
        predict = randomForestModel(xtrain, ytrain, xtest)
    else:
        print(i)
        train1, test1, vali = splitStore(train, test, i)
        xtrain, ytrain, xtest, xvali, yvali = reArrange(train1, test1, vali)
        xtrain, xtest, xvali = scaling(xtrain, xtest, xvali)
        predict = xgbModel(xtrain, ytrain, xtest, xvali, yvali)

    for i in predict:
        result.append(i)

1


KeyError: "['month'] not in index"